In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Reading and PreProcessing Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/flight-delays/flights.csv', low_memory=False)
df

In [ ]:
df = df[0:100000]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.value_counts('DIVERTED')

In [ ]:
df.corr()

In [ ]:
df[df.columns[1:]].corr()['ARRIVAL_DELAY'][:].sort_values(ascending=False)

JointPlot are a good method to visualise the realtion among features.

In [ ]:
sns.jointplot(data=df, x="SCHEDULED_ARRIVAL", y="ARRIVAL_TIME")


In [ ]:
df.columns

Drop Features with less correlation and more NaN values.

In [ ]:
df=df.drop(['YEAR','FLIGHT_NUMBER','AIRLINE','DISTANCE','TAIL_NUMBER','TAXI_OUT', 'SCHEDULED_TIME','DEPARTURE_TIME','WHEELS_OFF','ELAPSED_TIME', 'AIR_TIME','WHEELS_ON','DAY_OF_WEEK','TAXI_IN','CANCELLATION_REASON'], axis=1)

In [ ]:
df.columns

checking NULL values which needs to be imputed

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum()

Now filling the values with "mean" of that features, although it is not very efficient but for now it will work

In [ ]:
df=df.fillna(df.mean())

In [ ]:
corrMatrix = df.corr()
fig, ax = plt.subplots(figsize=(18,18))
sns.heatmap(corrMatrix, annot=True, ax=ax,fmt=".2g")
plt.show()

# Forming Target Column

Imputing the target column for the dataset. Condition being if the plane arrival delay is greater than 15. It is delayed.

In [ ]:
result=[]
for row in df['ARRIVAL_DELAY']:
    if row > 15:
        result.append(1)
    else:
        result.append(0)  
df['result'] = result

Lets see if our column has been added or not and how does the data look like

In [ ]:
print(df.columns[-1])
df['result'].head(10)

In [ ]:
#for me 1 is delayed and otherwise
df['result'].value_counts()

# Feature selection and Model Building

In [ ]:
df=df.drop(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ARRIVAL_TIME', 'ARRIVAL_DELAY'],axis=1)

In [ ]:
df=df.values
x, y = df[:,:-1], df[:,-1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [ ]:
print(x_train.shape+x_test.shape)

In [ ]:
#Scaling using sklearn predefined standardscalar
scaled_features = StandardScaler().fit_transform(x_train, x_test)

# Decision Tree Classifier.

as this problem is essentially a classification problem to decide which flight gets delayed which not.

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(x_train,y_train)
pred = clf.predict_proba(x_test)

In [ ]:
pred

In [ ]:
auc_score = roc_auc_score(y_test, pred[:,1])
print('AUC Score of Model is: {} %'.format(round(auc_score*100,2)))

# Prediction on Test Data

In [ ]:
result1 = confusion_matrix(y_test, pred[:,1])
print("Confusion Matrix:")
print(result1)
print('')
result2 = classification_report(y_test, pred[:,1])
print("Classification Report:",)
print (result2)
result3 = accuracy_score(y_test,pred[:,1])
print("Accuracy:",result3)


In [ ]:
print('Accuracy of Model on test data is: {} %'.format(round(result3*100,2)))

ROC curve visual plotting.

In [ ]:
plt.subplots(1, figsize=(10,6))
plt.title('Receiver Operating Characteristic - DecisionTree')
y_pred_proba = clf.predict_proba(x_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
plt.plot(fpr, tpr)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Thank You